# Biosimulators compatibility

## set-up

In [1]:
import os
import pandas as pd
from IPython.display import display_markdown
import shutil
import yaml

# Save the current working directory
cwd = os.getcwd()

# Change directory to the import utils 
os.chdir('../../')
import utils
path_to_sbml_folder = os.path.join(os.getcwd(), 'SBML')

# Change back to the original working directory
os.chdir(path_to_sbml_folder)

d1_plots_remote_dir = 'd1_plots'+ '_remote'
d1_plots_local_dir = 'd1_plots' + '_local'

sbml_file_name = 'LEMS_NML2_Ex9_FN.sbml'
sedml_file_name = 'LEMS_NML2_Ex9_FN_missing_xmlns.sedml' #xmlns:sbml missing

sbml_file_path = os.path.join(path_to_sbml_folder, sbml_file_name)
sedml_file_path = os.path.join(path_to_sbml_folder, sedml_file_name)

print('Path to SED-ML file:', sedml_file_path)
print('Path to SBML file:', sbml_file_path)

engines = utils.engines
types_dict = utils.types_dict

test_folder = 'tests'
output_dir = 'd1_plots'

d1_plots_remote_dir = os.path.join(test_folder, output_dir + '_remote')
d1_plots_local_dir = os.path.join(test_folder, output_dir + '_local')

print('d1 plots will be saved in:', d1_plots_remote_dir, 'and', d1_plots_local_dir)

Path to SED-ML file: c:\Users\prins\GitHub\SBMLShowcase\SBML\LEMS_NML2_Ex9_FN_missing_xmlns.sedml
Path to SBML file: c:\Users\prins\GitHub\SBMLShowcase\SBML\LEMS_NML2_Ex9_FN.sbml
d1 plots will be saved in: tests\d1_plots_remote and tests\d1_plots_local


In [2]:

#########################################################################################
# Run remotely
#########################################################################################

remote_output_dir = 'remote_results'
remote_output_dir = os.path.join(test_folder, remote_output_dir)

download_links_dict = dict()
for e in engines.keys():
    download_link = utils.run_biosimulators_remote(e, sedml_file_name, sbml_file_name)
    download_links_dict[e] = download_link

extract_dir_dict = dict()
for e, link in download_links_dict.items():
    extract_dir = utils.get_remote_results(e, link, remote_output_dir)
    extract_dir_dict[e] = extract_dir

results = dict()
for e, extract_dir in extract_dir_dict.items():
    status = ""
    error_message = ""
    exception_type = ""

    log_yml_path = utils.find_file_in_dir('log.yml', extract_dir)[0]
    if not log_yml_path:
        status = None
        error_message = 'log.yml not found'
        continue
    with open(log_yml_path) as f:
        log_yml_dict = yaml.safe_load(f)
        if log_yml_dict['status'] == 'SUCCEEDED':
            status = 'pass'
        elif log_yml_dict['status'] == 'FAILED':
            status = 'FAIL'
            exception = log_yml_dict['exception']
            error_message = exception['message']
            exception_type = exception['type'] 
        else:
            status = None
        results[e] = [status, error_message, exception_type] 

file_paths = utils.find_files(remote_output_dir, '.pdf')
utils.move_d1_files(file_paths, d1_plots_remote_dir)

# remove the remote results directory
if os.path.exists(remote_output_dir):
    shutil.rmtree(remote_output_dir)
    print('Removed ' + remote_output_dir + ' folder')

#########################################################################################
# Run locally
#########################################################################################

results_local = {}

output_folder = 'local_results'
local_output_dir = os.path.join(test_folder, output_folder)

for e in engines.keys():
    print('Running ' + e)
    local_output_dir_e = os.path.abspath(os.path.join(local_output_dir, e))
    print(local_output_dir_e)
    record = utils.run_biosimulators_docker(e, sedml_file_name, sbml_file_name, output_dir=local_output_dir_e)
    results_local[e] = record

file_paths = utils.find_files(local_output_dir, '.pdf')
print('file paths:', file_paths)
utils.move_d1_files(file_paths, d1_plots_local_dir)

# if it exists remove the output folder
if os.path.exists(local_output_dir):
    shutil.rmtree(local_output_dir)
    print('Removed ' + local_output_dir + ' folder')

#########################################################################################
# process results and save markdown table
#########################################################################################

results_table = utils.create_results_table(results, types_dict, sbml_file_name, sedml_file_name, engines, d1_plots_remote_dir)
results_table_local = utils.create_results_table(results_local, types_dict, sbml_file_name, sedml_file_name, engines, d1_plots_local_dir)

# rename cols to distinguish between local and remote results except for Engine column
results_table.columns = [str(col) + ' (R)' if col != 'Engine' else str(col) for col in results_table.columns]
results_table_local.columns = [str(col) + ' (L)' if col != 'Engine' else str(col) for col in results_table_local.columns]

# combine remote and local results
combined_results = pd.merge(results_table, results_table_local, on='Engine', how='outer')
combined_results = combined_results.reindex(columns=['Engine'] + sorted(combined_results.columns[1:]))

cols_order = ['Engine', 'pass / FAIL (R)', 'pass / FAIL (L)',\
               'Compat (R)', 'Compat (L)', \
               'Type (R)', \
               'Error (R)', 'Error (L)', \
               'd1 (R)', 'd1 (L)']

combined_results = combined_results[cols_order]

# add a table title
display_markdown('## Compatibility Biosimulator engines (L = local, R = remote)', raw=True)
display_markdown(combined_results.to_markdown(), raw=True)

pyNeuroML >>> DEBUG - Sim dict is: {'name': 'test', 'simulator': 'amici', 'simulatorVersion': '0.18.1', 'cpus': 1, 'memory': 8, 'maxTime': 20, 'envVars': [], 'purpose': 'academic', 'email': ''}
pyNeuroML >>> DEBUG - simulation_run_request is {"name":"test","simulator":"amici","simulatorVersion":"0.18.1","maxTime":20,"cpus":1,"memory":8,"purpose":"academic","email":"","envVars":[]}
pyNeuroML >>> DEBUG - data is:
{'file': ('LEMS_NML2_Ex9_FN_missing_xmlns.omex', <_io.BufferedReader name='LEMS_NML2_Ex9_FN_missing_xmlns.omex'>), 'simulationRun': (None, '{"name":"test","simulator":"amici","simulatorVersion":"0.18.1","maxTime":20,"cpus":1,"memory":8,"purpose":"academic","email":"","envVars":[]}')}
pyNeuroML >>> INFO - multipart encoded data is <MultipartEncoder: {'file': ('LEMS_NML2_Ex9_FN_missing_xmlns.omex', <_io.BufferedReader name='LEMS_NML2_Ex9_FN_missing_xmlns.omex'>), 'simulationRun': (None, '{"name":"test","simulator":"amici","simulatorVersion":"0.18.1","maxTime":20,"cpus":1,"memory":

Submitted LEMS_NML2_Ex9_FN_missing_xmlns.omex successfully with id: 66c4c7029f535bb92ffefba7
View: https://api.biosimulations.org/runs/66c4c7029f535bb92ffefba7
Downloads: https://api.biosimulations.org/results/66c4c7029f535bb92ffefba7/download
Logs: https://api.biosimulations.org/logs/66c4c7029f535bb92ffefba7?includeOutput=true


pyNeuroML >>> DEBUG - Sim dict is: {'name': 'test', 'simulator': 'brian2', 'simulatorVersion': '2.5.0.3', 'cpus': 1, 'memory': 8, 'maxTime': 20, 'envVars': [], 'purpose': 'academic', 'email': ''}
pyNeuroML >>> DEBUG - simulation_run_request is {"name":"test","simulator":"brian2","simulatorVersion":"2.5.0.3","maxTime":20,"cpus":1,"memory":8,"purpose":"academic","email":"","envVars":[]}
pyNeuroML >>> DEBUG - data is:
{'file': ('LEMS_NML2_Ex9_FN_missing_xmlns.omex', <_io.BufferedReader name='LEMS_NML2_Ex9_FN_missing_xmlns.omex'>), 'simulationRun': (None, '{"name":"test","simulator":"brian2","simulatorVersion":"2.5.0.3","maxTime":20,"cpus":1,"memory":8,"purpose":"academic","email":"","envVars":[]}')}
pyNeuroML >>> INFO - multipart encoded data is <MultipartEncoder: {'file': ('LEMS_NML2_Ex9_FN_missing_xmlns.omex', <_io.BufferedReader name='LEMS_NML2_Ex9_FN_missing_xmlns.omex'>), 'simulationRun': (None, '{"name":"test","simulator":"brian2","simulatorVersion":"2.5.0.3","maxTime":20,"cpus":1,"

Submitted LEMS_NML2_Ex9_FN_missing_xmlns.omex successfully with id: 66c4c70493c284abde6a09a2
View: https://api.biosimulations.org/runs/66c4c70493c284abde6a09a2
Downloads: https://api.biosimulations.org/results/66c4c70493c284abde6a09a2/download
Logs: https://api.biosimulations.org/logs/66c4c70493c284abde6a09a2?includeOutput=true


pyNeuroML >>> DEBUG - Sim dict is: {'name': 'test', 'simulator': 'bionetgen', 'simulatorVersion': '2.8.0', 'cpus': 1, 'memory': 8, 'maxTime': 20, 'envVars': [], 'purpose': 'academic', 'email': ''}
pyNeuroML >>> DEBUG - simulation_run_request is {"name":"test","simulator":"bionetgen","simulatorVersion":"2.8.0","maxTime":20,"cpus":1,"memory":8,"purpose":"academic","email":"","envVars":[]}
pyNeuroML >>> DEBUG - data is:
{'file': ('LEMS_NML2_Ex9_FN_missing_xmlns.omex', <_io.BufferedReader name='LEMS_NML2_Ex9_FN_missing_xmlns.omex'>), 'simulationRun': (None, '{"name":"test","simulator":"bionetgen","simulatorVersion":"2.8.0","maxTime":20,"cpus":1,"memory":8,"purpose":"academic","email":"","envVars":[]}')}
pyNeuroML >>> INFO - multipart encoded data is <MultipartEncoder: {'file': ('LEMS_NML2_Ex9_FN_missing_xmlns.omex', <_io.BufferedReader name='LEMS_NML2_Ex9_FN_missing_xmlns.omex'>), 'simulationRun': (None, '{"name":"test","simulator":"bionetgen","simulatorVersion":"2.8.0","maxTime":20,"cpus"

...

Submitted LEMS_NML2_Ex9_FN_missing_xmlns.omex successfully with id: 66c4c72893c284abde6a0a1a
View: https://api.biosimulations.org/runs/66c4c72893c284abde6a0a1a
Downloads: https://api.biosimulations.org/results/66c4c72893c284abde6a0a1a/download
Logs: https://api.biosimulations.org/logs/66c4c72893c284abde6a0a1a?includeOutput=true


pyNeuroML >>> DEBUG - Sim dict is: {'name': 'test', 'simulator': 'xpp', 'simulatorVersion': '8.0', 'cpus': 1, 'memory': 8, 'maxTime': 20, 'envVars': [], 'purpose': 'academic', 'email': ''}
pyNeuroML >>> DEBUG - simulation_run_request is {"name":"test","simulator":"xpp","simulatorVersion":"8.0","maxTime":20,"cpus":1,"memory":8,"purpose":"academic","email":"","envVars":[]}
pyNeuroML >>> DEBUG - data is:
{'file': ('LEMS_NML2_Ex9_FN_missing_xmlns.omex', <_io.BufferedReader name='LEMS_NML2_Ex9_FN_missing_xmlns.omex'>), 'simulationRun': (None, '{"name":"test","simulator":"xpp","simulatorVersion":"8.0","maxTime":20,"cpus":1,"memory":8,"purpose":"academic","email":"","envVars":[]}')}
pyNeuroML >>> INFO - multipart encoded data is <MultipartEncoder: {'file': ('LEMS_NML2_Ex9_FN_missing_xmlns.omex', <_io.BufferedReader name='LEMS_NML2_Ex9_FN_missing_xmlns.omex'>), 'simulationRun': (None, '{"name":"test","simulator":"xpp","simulatorVersion":"8.0","maxTime":20,"cpus":1,"memory":8,"purpose":"academi

Submitted LEMS_NML2_Ex9_FN_missing_xmlns.omex successfully with id: 66c4c72a9f535bb92ffefc37
View: https://api.biosimulations.org/runs/66c4c72a9f535bb92ffefc37
Downloads: https://api.biosimulations.org/results/66c4c72a9f535bb92ffefc37/download
Logs: https://api.biosimulations.org/logs/66c4c72a9f535bb92ffefc37?includeOutput=true
Moving tests\remote_results\amici\outputs\tmp445545\d1.pdf to tests\d1_plots_remote\amici_d1.pdf
Moving tests\remote_results\bionetgen\outputs\tmp713444\d1.pdf to tests\d1_plots_remote\bionetgen_d1.pdf
Moving tests\remote_results\boolnet\outputs\tmp993713\d1.pdf to tests\d1_plots_remote\boolnet_d1.pdf
Moving tests\remote_results\cbmpy\outputs\tmp924986\d1.pdf to tests\d1_plots_remote\cbmpy_d1.pdf
Moving tests\remote_results\cobrapy\outputs\tmp796378\d1.pdf to tests\d1_plots_remote\cobrapy_d1.pdf
Moving tests\remote_results\copasi\outputs\tmp39638\d1.pdf to tests\d1_plots_remote\copasi_d1.pdf
Moving tests\remote_results\gillespy2\outputs\tmp785210\d1.pdf to tests

## Compatibility Biosimulator engines (L = local, R = remote)

|    | Engine                                                                                                                                     | pass / FAIL (R)                                                                                                                                                            | pass / FAIL (L)                                                                                                                                                            | Compat (R)                                                                                                                                                                                                                                                                                                                              | Compat (L)                                                                                                                                                                                                                                                                                                                              | Type (R)                                                               | Error (R)                                                                                                                                                                                                                                                                                                                                                                                                                                                          | Error (L)                                                                                                                                                                                            | d1 (R)                                                     | d1 (L)                                                    |
|---:|:-------------------------------------------------------------------------------------------------------------------------------------------|:---------------------------------------------------------------------------------------------------------------------------------------------------------------------------|:---------------------------------------------------------------------------------------------------------------------------------------------------------------------------|:----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|:----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|:-----------------------------------------------------------------------|:-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|:-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|:-----------------------------------------------------------|:----------------------------------------------------------|
|  0 | <details><summary>amici</summary>https://docs.biosimulators.org/Biosimulators_AMICI/<br></details>                                         | <img src=https://via.placeholder.com/15/00dd00/00dd00.png/> pass                                                                                                           | <img src=https://via.placeholder.com/15/00dd00/00dd00.png/> pass                                                                                                           | <img src=https://via.placeholder.com/15/00dd00/00dd00.png/><details><summary>pass</summary>The file extensions suggest the input file types are '['SBML', 'SED-ML']'. These are compatible with amici</details>                                                                                                                         | <img src=https://via.placeholder.com/15/00dd00/00dd00.png/><details><summary>pass</summary>The file extensions suggest the input file types are '['SBML', 'SED-ML']'. These are compatible with amici</details>                                                                                                                         |                                                                        |                                                                                                                                                                                                                                                                                                                                                                                                                                                                    |                                                                                                                                                                                                      | <a href="tests\d1_plots_remote\amici_d1.pdf">plot</a>      | <a href="tests\d1_plots_local\amici_d1.pdf">plot</a>      |
|  1 | <details><summary>bionetgen</summary>https://docs.biosimulators.org/Biosimulators_BioNetGen/<br></details>                                 | <span style="color:darkred;">                                                                      <img src=https://via.placeholder.com/15/dd0000/dd0000.png/> FAIL</span> | <span style="color:darkred;">                                                                      <img src=https://via.placeholder.com/15/dd0000/dd0000.png/> FAIL</span> | <span style="color:darkred;"><img src=https://via.placeholder.com/15/dd0000/dd0000.png/> <details><summary>FAIL</summary>The file extensions suggest the input file types are '['SBML', 'SED-ML']'. These are not compatible with bionetgen. The following file types will be compatible ['BNGL', 'SED-ML']</details></span>            | <span style="color:darkred;"><img src=https://via.placeholder.com/15/dd0000/dd0000.png/> <details><summary>FAIL</summary>The file extensions suggest the input file types are '['SBML', 'SED-ML']'. These are not compatible with bionetgen. The following file types will be compatible ['BNGL', 'SED-ML']</details></span>            | <details><summary>CAEE</summary>CombineArchiveExecutionError</details> | <details><summary>Details</summary><span style="color:red;">The COMBINE/OMEX did not execute successfully:<br><br>  The SED document did not execute successfully:<br>  <br>    Language for model `net1` is not supported.<br>      - Model language `urn:sedml:language:sbml` is not supported. Models must be in BNGL format (e.g., `sed:model/@language` must match `^urn:sedml:language:bngl(\.$)` such as `urn:sedml:language:bngl`).</details>              | <details><summary>Details</summary>```Command '-i /root/in/LEMS_NML2_Ex9_FN_missing_xmlns.omex -o /root/out' in image 'ghcr.io/biosimulators/bionetgen' returned non-zero exit status 1```</details> | <a href="tests\d1_plots_remote\bionetgen_d1.pdf">plot</a>  | <a href="tests\d1_plots_local\bionetgen_d1.pdf">plot</a>  |
|  2 | <details><summary>boolnet</summary>https://docs.biosimulators.org/Biosimulators_BoolNet/<br></details>                                     | <span style="color:darkred;">                                                                      <img src=https://via.placeholder.com/15/dd0000/dd0000.png/> FAIL</span> | <span style="color:darkred;">                                                                      <img src=https://via.placeholder.com/15/dd0000/dd0000.png/> FAIL</span> | <span style="color:darkred;"><img src=https://via.placeholder.com/15/dd0000/dd0000.png/> <details><summary>FAIL</summary>The file extensions suggest the input file types are '['SBML', 'SED-ML']'. These are not compatible with boolnet. The following file types will be compatible ['SBML-qual', 'SED-ML']</details></span>         | <span style="color:darkred;"><img src=https://via.placeholder.com/15/dd0000/dd0000.png/> <details><summary>FAIL</summary>The file extensions suggest the input file types are '['SBML', 'SED-ML']'. These are not compatible with boolnet. The following file types will be compatible ['SBML-qual', 'SED-ML']</details></span>         | <details><summary>CAEE</summary>CombineArchiveExecutionError</details> | <details><summary>Details</summary><span style="color:red;">The COMBINE/OMEX did not execute successfully:<br><br>  The SED document did not execute successfully:<br>  <br>    Simulation `sim1` is invalid.<br>      - Number of points (20000) must be equal to the difference between the output end (200.0) and start times (0.0).</details>                                                                                                                  | <details><summary>Details</summary>```Command '-i /root/in/LEMS_NML2_Ex9_FN_missing_xmlns.omex -o /root/out' in image 'ghcr.io/biosimulators/boolnet' returned non-zero exit status 1```</details>   | <a href="tests\d1_plots_remote\boolnet_d1.pdf">plot</a>    | <a href="tests\d1_plots_local\boolnet_d1.pdf">plot</a>    |
|  3 | <details><summary>brian2</summary>https://docs.biosimulators.org/Biosimulators_pyNeuroML/<br></details>                                    | <span style="color:darkred;">                                                                      <img src=https://via.placeholder.com/15/dd0000/dd0000.png/> FAIL</span> | <span style="color:darkred;">                                                                      <img src=https://via.placeholder.com/15/dd0000/dd0000.png/> FAIL</span> | <img src=https://via.placeholder.com/15/00dd00/00dd00.png/><details><summary>pass</summary>The file extensions suggest the input file types are '['SBML', 'SED-ML']'. These are compatible with brian2</details>                                                                                                                        | <img src=https://via.placeholder.com/15/00dd00/00dd00.png/><details><summary>pass</summary>The file extensions suggest the input file types are '['SBML', 'SED-ML']'. These are compatible with brian2</details>                                                                                                                        | <details><summary>MNFE</summary>ModuleNotFoundError</details>          | <details><summary>Details</summary>No module named 'libsbml'</details>                                                                                                                                                                                                                                                                                                                                                                                             | <details><summary>Details</summary>```Command '-i /root/in/LEMS_NML2_Ex9_FN_missing_xmlns.omex -o /root/out' in image 'ghcr.io/biosimulators/brian2' returned non-zero exit status 1```</details>    |                                                            |                                                           |
|  4 | <details><summary>cbmpy</summary>https://docs.biosimulators.org/Biosimulators_CBMPy/<br></details>                                         | <span style="color:darkred;">                                                                      <img src=https://via.placeholder.com/15/dd0000/dd0000.png/> FAIL</span> | <span style="color:darkred;">                                                                      <img src=https://via.placeholder.com/15/dd0000/dd0000.png/> FAIL</span> | <img src=https://via.placeholder.com/15/00dd00/00dd00.png/><details><summary>pass</summary>The file extensions suggest the input file types are '['SBML', 'SED-ML']'. These are compatible with cbmpy</details>                                                                                                                         | <img src=https://via.placeholder.com/15/00dd00/00dd00.png/><details><summary>pass</summary>The file extensions suggest the input file types are '['SBML', 'SED-ML']'. These are compatible with cbmpy</details>                                                                                                                         | <details><summary>CAEE</summary>CombineArchiveExecutionError</details> | <details><summary>Details</summary><span style="color:red;">The COMBINE/OMEX did not execute successfully:<br><br>  The SED document did not execute successfully:<br>  <br>    UniformTimeCourseSimulation `sim1` is not supported.<br>      - Simulation sim1 of type `UniformTimeCourseSimulation` is not supported. Simulation must be an instance of one of the following:<br>          - SteadyStateSimulation</details>                                     | <details><summary>Details</summary>```Command '-i /root/in/LEMS_NML2_Ex9_FN_missing_xmlns.omex -o /root/out' in image 'ghcr.io/biosimulators/cbmpy' returned non-zero exit status 1```</details>     | <a href="tests\d1_plots_remote\cbmpy_d1.pdf">plot</a>      | <a href="tests\d1_plots_local\cbmpy_d1.pdf">plot</a>      |
|  5 | <details><summary>cobrapy</summary>https://docs.biosimulators.org/Biosimulators_COBRApy/<br>Only allows steady state simulations</details> | <span style="color:darkred;">                                                                      <img src=https://via.placeholder.com/15/dd0000/dd0000.png/> FAIL</span> | <span style="color:darkred;">                                                                      <img src=https://via.placeholder.com/15/dd0000/dd0000.png/> FAIL</span> | <img src=https://via.placeholder.com/15/00dd00/00dd00.png/><details><summary>pass</summary>The file extensions suggest the input file types are '['SBML', 'SED-ML']'. These are compatible with cobrapy</details>                                                                                                                       | <img src=https://via.placeholder.com/15/00dd00/00dd00.png/><details><summary>pass</summary>The file extensions suggest the input file types are '['SBML', 'SED-ML']'. These are compatible with cobrapy</details>                                                                                                                       | <details><summary>CAEE</summary>CombineArchiveExecutionError</details> | <details><summary>Details</summary><span style="color:red;">The COMBINE/OMEX did not execute successfully:<br><br>  The SED document did not execute successfully:<br>  <br>    UniformTimeCourseSimulation `sim1` is not supported.<br>      - Simulation sim1 of type `UniformTimeCourseSimulation` is not supported. Simulation must be an instance of one of the following:<br>          - SteadyStateSimulation</details>                                     | <details><summary>Details</summary>```Command '-i /root/in/LEMS_NML2_Ex9_FN_missing_xmlns.omex -o /root/out' in image 'ghcr.io/biosimulators/cobrapy' returned non-zero exit status 1```</details>   | <a href="tests\d1_plots_remote\cobrapy_d1.pdf">plot</a>    | <a href="tests\d1_plots_local\cobrapy_d1.pdf">plot</a>    |
|  6 | <details><summary>copasi</summary>https://docs.biosimulators.org/Biosimulators_COPASI/<br></details>                                       | <span style="color:darkred;">                                                                      <img src=https://via.placeholder.com/15/dd0000/dd0000.png/> FAIL</span> | <span style="color:darkred;">                                                                      <img src=https://via.placeholder.com/15/dd0000/dd0000.png/> FAIL</span> | <img src=https://via.placeholder.com/15/00dd00/00dd00.png/><details><summary>pass</summary>The file extensions suggest the input file types are '['SBML', 'SED-ML']'. These are compatible with copasi</details>                                                                                                                        | <img src=https://via.placeholder.com/15/00dd00/00dd00.png/><details><summary>pass</summary>The file extensions suggest the input file types are '['SBML', 'SED-ML']'. These are compatible with copasi</details>                                                                                                                        | <details><summary>CAEE</summary>CombineArchiveExecutionError</details> | <details><summary>Details</summary><span style="color:red;">The COMBINE/OMEX did not execute successfully:<br><br>  The SED document did not execute successfully:<br>  <br>    could not convert string to float: 'Values[V]'</details>                                                                                                                                                                                                                           | <details><summary>Details</summary>```Command '-i /root/in/LEMS_NML2_Ex9_FN_missing_xmlns.omex -o /root/out' in image 'ghcr.io/biosimulators/copasi' returned non-zero exit status 1```</details>    | <a href="tests\d1_plots_remote\copasi_d1.pdf">plot</a>     | <a href="tests\d1_plots_local\copasi_d1.pdf">plot</a>     |
|  7 | <details><summary>gillespy2</summary>https://docs.biosimulators.org/Biosimulators_GillesPy2/<br></details>                                 | <img src=https://via.placeholder.com/15/00dd00/00dd00.png/> pass                                                                                                           | <img src=https://via.placeholder.com/15/00dd00/00dd00.png/> pass                                                                                                           | <img src=https://via.placeholder.com/15/00dd00/00dd00.png/><details><summary>pass</summary>The file extensions suggest the input file types are '['SBML', 'SED-ML']'. These are compatible with gillespy2</details>                                                                                                                     | <img src=https://via.placeholder.com/15/00dd00/00dd00.png/><details><summary>pass</summary>The file extensions suggest the input file types are '['SBML', 'SED-ML']'. These are compatible with gillespy2</details>                                                                                                                     |                                                                        |                                                                                                                                                                                                                                                                                                                                                                                                                                                                    |                                                                                                                                                                                                      | <a href="tests\d1_plots_remote\gillespy2_d1.pdf">plot</a>  | <a href="tests\d1_plots_local\gillespy2_d1.pdf">plot</a>  |
|  8 | <details><summary>ginsim</summary>https://docs.biosimulators.org/Biosimulators_GINsim/<br></details>                                       | <span style="color:darkred;">                                                                      <img src=https://via.placeholder.com/15/dd0000/dd0000.png/> FAIL</span> | <span style="color:darkred;">                                                                      <img src=https://via.placeholder.com/15/dd0000/dd0000.png/> FAIL</span> | <span style="color:darkred;"><img src=https://via.placeholder.com/15/dd0000/dd0000.png/> <details><summary>FAIL</summary>The file extensions suggest the input file types are '['SBML', 'SED-ML']'. These are not compatible with ginsim. The following file types will be compatible ['SBML-qual', 'SED-ML']</details></span>          | <span style="color:darkred;"><img src=https://via.placeholder.com/15/dd0000/dd0000.png/> <details><summary>FAIL</summary>The file extensions suggest the input file types are '['SBML', 'SED-ML']'. These are not compatible with ginsim. The following file types will be compatible ['SBML-qual', 'SED-ML']</details></span>          | <details><summary>CAEE</summary>CombineArchiveExecutionError</details> | <details><summary>Details</summary><span style="color:red;">The COMBINE/OMEX did not execute successfully:<br><br>  The SED document did not execute successfully:<br>  <br>    Simulation `sim1` is invalid.<br>      - The interval between the output start and time time must be an integer multiple of the number of steps, not `0.01`:<br>          Output start time: 0.0<br>          Output end time: 200.0<br>          Number of steps: 20000</details> | <details><summary>Details</summary>```Command '-i /root/in/LEMS_NML2_Ex9_FN_missing_xmlns.omex -o /root/out' in image 'ghcr.io/biosimulators/ginsim' returned non-zero exit status 1```</details>    | <a href="tests\d1_plots_remote\ginsim_d1.pdf">plot</a>     | <a href="tests\d1_plots_local\ginsim_d1.pdf">plot</a>     |
|  9 | <details><summary>libsbmlsim</summary>https://docs.biosimulators.org/Biosimulators_LibSBMLSim/<br></details>                               | <img src=https://via.placeholder.com/15/00dd00/00dd00.png/> pass                                                                                                           | <img src=https://via.placeholder.com/15/00dd00/00dd00.png/> pass                                                                                                           | <img src=https://via.placeholder.com/15/00dd00/00dd00.png/><details><summary>pass</summary>The file extensions suggest the input file types are '['SBML', 'SED-ML']'. These are compatible with libsbmlsim</details>                                                                                                                    | <img src=https://via.placeholder.com/15/00dd00/00dd00.png/><details><summary>pass</summary>The file extensions suggest the input file types are '['SBML', 'SED-ML']'. These are compatible with libsbmlsim</details>                                                                                                                    |                                                                        |                                                                                                                                                                                                                                                                                                                                                                                                                                                                    |                                                                                                                                                                                                      | <a href="tests\d1_plots_remote\libsbmlsim_d1.pdf">plot</a> | <a href="tests\d1_plots_local\libsbmlsim_d1.pdf">plot</a> |
| 10 | <details><summary>masspy</summary>https://docs.biosimulators.org/Biosimulators_MASSpy/<br></details>                                       | <span style="color:darkred;">                                                                      <img src=https://via.placeholder.com/15/dd0000/dd0000.png/> FAIL</span> | <span style="color:darkred;">                                                                      <img src=https://via.placeholder.com/15/dd0000/dd0000.png/> FAIL</span> | <img src=https://via.placeholder.com/15/00dd00/00dd00.png/><details><summary>pass</summary>The file extensions suggest the input file types are '['SBML', 'SED-ML']'. These are compatible with masspy</details>                                                                                                                        | <img src=https://via.placeholder.com/15/00dd00/00dd00.png/><details><summary>pass</summary>The file extensions suggest the input file types are '['SBML', 'SED-ML']'. These are compatible with masspy</details>                                                                                                                        | <details><summary>CAEE</summary>CombineArchiveExecutionError</details> | <details><summary>Details</summary><span style="color:red;">The COMBINE/OMEX did not execute successfully:<br><br>  The SED document did not execute successfully:<br>  <br>    Something went wrong reading the SBML model. Most likely the SBML model is not valid. Please check that your model is valid using the `mass.io.sbml.validate_sbml_model` function or via the online validator at http://sbml.org/validator .<br>    	`(model, errors) = validate_sbml_model(filename)`<br>    If the model is valid and cannot be read please open an issue at https://github.com/SBRG/masspy/issues .</details>                                                                                                                                                                                                                                                                                                                                                                                                                                                                    | <details><summary>Details</summary>```Command '-i /root/in/LEMS_NML2_Ex9_FN_missing_xmlns.omex -o /root/out' in image 'ghcr.io/biosimulators/masspy' returned non-zero exit status 1```</details>    | <a href="tests\d1_plots_remote\masspy_d1.pdf">plot</a>     | <a href="tests\d1_plots_local\masspy_d1.pdf">plot</a>     |
| 11 | <details><summary>netpyne</summary>https://docs.biosimulators.org/Biosimulators_pyNeuroML/<br></details>                                   | <span style="color:darkred;">                                                                      <img src=https://via.placeholder.com/15/dd0000/dd0000.png/> FAIL</span> | <span style="color:darkred;">                                                                      <img src=https://via.placeholder.com/15/dd0000/dd0000.png/> FAIL</span> | <img src=https://via.placeholder.com/15/00dd00/00dd00.png/><details><summary>pass</summary>The file extensions suggest the input file types are '['SBML', 'SED-ML']'. These are compatible with netpyne</details>                                                                                                                       | <img src=https://via.placeholder.com/15/00dd00/00dd00.png/><details><summary>pass</summary>The file extensions suggest the input file types are '['SBML', 'SED-ML']'. These are compatible with netpyne</details>                                                                                                                       | <details><summary>MNFE</summary>ModuleNotFoundError</details>          | <details><summary>Details</summary>No module named 'libsbml'</details>                                                                                                                                                                                                                                                                                                                                                                                             | <details><summary>Details</summary>```Command '-i /root/in/LEMS_NML2_Ex9_FN_missing_xmlns.omex -o /root/out' in image 'ghcr.io/biosimulators/netpyne' returned non-zero exit status 1```</details>   |                                                            |                                                           |
| 12 | <details><summary>neuron</summary>https://docs.biosimulators.org/Biosimulators_pyNeuroML/<br></details>                                    | <span style="color:darkred;">                                                                      <img src=https://via.placeholder.com/15/dd0000/dd0000.png/> FAIL</span> | <span style="color:darkred;">                                                                      <img src=https://via.placeholder.com/15/dd0000/dd0000.png/> FAIL</span> | <span style="color:darkred;"><img src=https://via.placeholder.com/15/dd0000/dd0000.png/> <details><summary>FAIL</summary>The file extensions suggest the input file types are '['SBML', 'SED-ML']'. These are not compatible with neuron. The following file types will be compatible ['NeuroML', 'LEMS', 'SED-ML']</details></span>    | <span style="color:darkred;"><img src=https://via.placeholder.com/15/dd0000/dd0000.png/> <details><summary>FAIL</summary>The file extensions suggest the input file types are '['SBML', 'SED-ML']'. These are not compatible with neuron. The following file types will be compatible ['NeuroML', 'LEMS', 'SED-ML']</details></span>    | <details><summary>MNFE</summary>ModuleNotFoundError</details>          | <details><summary>Details</summary>No module named 'libsbml'</details>                                                                                                                                                                                                                                                                                                                                                                                             | <details><summary>Details</summary>```Command '-i /root/in/LEMS_NML2_Ex9_FN_missing_xmlns.omex -o /root/out' in image 'ghcr.io/biosimulators/neuron' returned non-zero exit status 1```</details>    |                                                            |                                                           |
| 13 | <details><summary>opencor</summary>https://docs.biosimulators.org/Biosimulators_OpenCOR/<br></details>                                     | <span style="color:darkred;">                                                                      <img src=https://via.placeholder.com/15/dd0000/dd0000.png/> FAIL</span> | <span style="color:darkred;">                                                                      <img src=https://via.placeholder.com/15/dd0000/dd0000.png/> FAIL</span> | <span style="color:darkred;"><img src=https://via.placeholder.com/15/dd0000/dd0000.png/> <details><summary>FAIL</summary>The file extensions suggest the input file types are '['SBML', 'SED-ML']'. These are not compatible with opencor. The following file types will be compatible ['CellML', 'SED-ML']</details></span>            | <span style="color:darkred;"><img src=https://via.placeholder.com/15/dd0000/dd0000.png/> <details><summary>FAIL</summary>The file extensions suggest the input file types are '['SBML', 'SED-ML']'. These are not compatible with opencor. The following file types will be compatible ['CellML', 'SED-ML']</details></span>            | <details><summary>MNFE</summary>ModuleNotFoundError</details>          | <details><summary>Details</summary>No module named 'libsbml'</details>                                                                                                                                                                                                                                                                                                                                                                                             | <details><summary>Details</summary>```Command '-i /root/in/LEMS_NML2_Ex9_FN_missing_xmlns.omex -o /root/out' in image 'ghcr.io/biosimulators/opencor' returned non-zero exit status 1```</details>   |                                                            |                                                           |
| 14 | <details><summary>pyneuroml</summary>https://docs.biosimulators.org/Biosimulators_pyNeuroML/<br></details>                                 | <span style="color:darkred;">                                                                      <img src=https://via.placeholder.com/15/dd0000/dd0000.png/> FAIL</span> | <span style="color:darkred;">                                                                      <img src=https://via.placeholder.com/15/dd0000/dd0000.png/> FAIL</span> | <span style="color:darkred;"><img src=https://via.placeholder.com/15/dd0000/dd0000.png/> <details><summary>FAIL</summary>The file extensions suggest the input file types are '['SBML', 'SED-ML']'. These are not compatible with pyneuroml. The following file types will be compatible ['NeuroML', 'LEMS', 'SED-ML']</details></span> | <span style="color:darkred;"><img src=https://via.placeholder.com/15/dd0000/dd0000.png/> <details><summary>FAIL</summary>The file extensions suggest the input file types are '['SBML', 'SED-ML']'. These are not compatible with pyneuroml. The following file types will be compatible ['NeuroML', 'LEMS', 'SED-ML']</details></span> | <details><summary>MNFE</summary>ModuleNotFoundError</details>          | <details><summary>Details</summary>No module named 'libsbml'</details>                                                                                                                                                                                                                                                                                                                                                                                             | <details><summary>Details</summary>```Command '-i /root/in/LEMS_NML2_Ex9_FN_missing_xmlns.omex -o /root/out' in image 'ghcr.io/biosimulators/pyneuroml' returned non-zero exit status 1```</details> |                                                            |                                                           |
| 15 | <details><summary>pysces</summary>https://docs.biosimulators.org/Biosimulators_PySCeS/<br></details>                                       | <span style="color:darkred;">                                                                      <img src=https://via.placeholder.com/15/dd0000/dd0000.png/> FAIL</span> | <span style="color:darkred;">                                                                      <img src=https://via.placeholder.com/15/dd0000/dd0000.png/> FAIL</span> | <img src=https://via.placeholder.com/15/00dd00/00dd00.png/><details><summary>pass</summary>The file extensions suggest the input file types are '['SBML', 'SED-ML']'. These are compatible with pysces</details>                                                                                                                        | <img src=https://via.placeholder.com/15/00dd00/00dd00.png/><details><summary>pass</summary>The file extensions suggest the input file types are '['SBML', 'SED-ML']'. These are compatible with pysces</details>                                                                                                                        | <details><summary>CAEE</summary>CombineArchiveExecutionError</details> | <details><summary>Details</summary><span style="color:red;">The COMBINE/OMEX did not execute successfully:<br><br>  The SED document did not execute successfully:<br>  <br>    Model at /tmp/tmpr1stywf8/./LEMS_NML2_Ex9_FN.sbml could not be imported:<br>      <br>      File /tmp/tmpr1stywf8/./LEMS_NML2_Ex9_FN.sbml.xml does not exist</details>                                                                                                             | <details><summary>Details</summary>```Command '-i /root/in/LEMS_NML2_Ex9_FN_missing_xmlns.omex -o /root/out' in image 'ghcr.io/biosimulators/pysces' returned non-zero exit status 1```</details>    | <a href="tests\d1_plots_remote\pysces_d1.pdf">plot</a>     | <a href="tests\d1_plots_local\pysces_d1.pdf">plot</a>     |
| 16 | <details><summary>rbapy</summary>https://docs.biosimulators.org/Biosimulators_RBApy/<br></details>                                         | <span style="color:darkred;">                                                                      <img src=https://via.placeholder.com/15/dd0000/dd0000.png/> FAIL</span> | <span style="color:darkred;">                                                                      <img src=https://via.placeholder.com/15/dd0000/dd0000.png/> FAIL</span> | <span style="color:darkred;"><img src=https://via.placeholder.com/15/dd0000/dd0000.png/> <details><summary>FAIL</summary>The file extensions suggest the input file types are '['SBML', 'SED-ML']'. These are not compatible with rbapy. The following file types will be compatible ['RBApy', 'SED-ML']</details></span>               | <span style="color:darkred;"><img src=https://via.placeholder.com/15/dd0000/dd0000.png/> <details><summary>FAIL</summary>The file extensions suggest the input file types are '['SBML', 'SED-ML']'. These are not compatible with rbapy. The following file types will be compatible ['RBApy', 'SED-ML']</details></span>               | <details><summary>CAEE</summary>CombineArchiveExecutionError</details> | <details><summary>Details</summary><span style="color:red;">The COMBINE/OMEX did not execute successfully:<br><br>  The SED document did not execute successfully:<br>  <br>    Language for model `net1` is not supported.<br>      - Model language `urn:sedml:language:sbml` is not supported. Models must be in RBA format (e.g., `sed:model/@language` must match `^urn:sedml:language:rba(\.$)` such as `urn:sedml:language:rba`).</details>                 | <details><summary>Details</summary>```Command '-i /root/in/LEMS_NML2_Ex9_FN_missing_xmlns.omex -o /root/out' in image 'ghcr.io/biosimulators/rbapy' returned non-zero exit status 1```</details>     | <a href="tests\d1_plots_remote\rbapy_d1.pdf">plot</a>      | <a href="tests\d1_plots_local\rbapy_d1.pdf">plot</a>      |
| 17 | <details><summary>smoldyn</summary>https://smoldyn.readthedocs.io/en/latest/python/api.html#sed-ml-combine-biosimulators-api<br></details> | <span style="color:darkred;">                                                                      <img src=https://via.placeholder.com/15/dd0000/dd0000.png/> FAIL</span> | <span style="color:darkred;">                                                                      <img src=https://via.placeholder.com/15/dd0000/dd0000.png/> FAIL</span> | <span style="color:darkred;"><img src=https://via.placeholder.com/15/dd0000/dd0000.png/> <details><summary>FAIL</summary>smoldyn compatible file types unknown.</details></span>                                                                                                                                                        | <span style="color:darkred;"><img src=https://via.placeholder.com/15/dd0000/dd0000.png/> <details><summary>FAIL</summary>smoldyn compatible file types unknown.</details></span>                                                                                                                                                        | <details><summary>MNFE</summary>ModuleNotFoundError</details>          | <details><summary>Details</summary>No module named 'libsbml'</details>                                                                                                                                                                                                                                                                                                                                                                                             | <details><summary>Details</summary>```Command '-i /root/in/LEMS_NML2_Ex9_FN_missing_xmlns.omex -o /root/out' in image 'ghcr.io/biosimulators/smoldyn' returned non-zero exit status 1```</details>   |                                                            |                                                           |
| 18 | <details><summary>tellurium</summary>https://docs.biosimulators.org/Biosimulators_tellurium/<br></details>                                 | <img src=https://via.placeholder.com/15/00dd00/00dd00.png/> pass                                                                                                           | <img src=https://via.placeholder.com/15/00dd00/00dd00.png/> pass                                                                                                           | <img src=https://via.placeholder.com/15/00dd00/00dd00.png/><details><summary>pass</summary>The file extensions suggest the input file types are '['SBML', 'SED-ML']'. These are compatible with tellurium</details>                                                                                                                     | <img src=https://via.placeholder.com/15/00dd00/00dd00.png/><details><summary>pass</summary>The file extensions suggest the input file types are '['SBML', 'SED-ML']'. These are compatible with tellurium</details>                                                                                                                     |                                                                        |                                                                                                                                                                                                                                                                                                                                                                                                                                                                    |                                                                                                                                                                                                      | <a href="tests\d1_plots_remote\tellurium_d1.pdf">plot</a>  | <a href="tests\d1_plots_local\tellurium_d1.pdf">plot</a>  |
| 19 | <details><summary>vcell</summary>https://github.com/virtualcell/vcell<br></details>                                                        | <img src=https://via.placeholder.com/15/00dd00/00dd00.png/> pass                                                                                                           | <span style="color:darkred;">                                                                      <img src=https://via.placeholder.com/15/dd0000/dd0000.png/> FAIL</span> | <img src=https://via.placeholder.com/15/00dd00/00dd00.png/><details><summary>pass</summary>The file extensions suggest the input file types are '['SBML', 'SED-ML']'. These are compatible with vcell</details>                                                                                                                         | <img src=https://via.placeholder.com/15/00dd00/00dd00.png/><details><summary>pass</summary>The file extensions suggest the input file types are '['SBML', 'SED-ML']'. These are compatible with vcell</details>                                                                                                                         |                                                                        |                                                                                                                                                                                                                                                                                                                                                                                                                                                                    | <details><summary>Details</summary>```Command '-i /root/in/LEMS_NML2_Ex9_FN_missing_xmlns.omex -o /root/out' in image 'ghcr.io/biosimulators/vcell' returned non-zero exit status 1```</details>     |                                                            |                                                           |
| 20 | <details><summary>xpp</summary>https://docs.biosimulators.org/Biosimulators_XPP/<br></details>                                             | <span style="color:darkred;">                                                                      <img src=https://via.placeholder.com/15/dd0000/dd0000.png/> FAIL</span> | <span style="color:darkred;">                                                                      <img src=https://via.placeholder.com/15/dd0000/dd0000.png/> FAIL</span> | <span style="color:darkred;"><img src=https://via.placeholder.com/15/dd0000/dd0000.png/> <details><summary>FAIL</summary>The file extensions suggest the input file types are '['SBML', 'SED-ML']'. These are not compatible with xpp. The following file types will be compatible ['XPP', 'SED-ML']</details></span>                   | <span style="color:darkred;"><img src=https://via.placeholder.com/15/dd0000/dd0000.png/> <details><summary>FAIL</summary>The file extensions suggest the input file types are '['SBML', 'SED-ML']'. These are not compatible with xpp. The following file types will be compatible ['XPP', 'SED-ML']</details></span>                   | <details><summary>MNFE</summary>ModuleNotFoundError</details>          | <details><summary>Details</summary>No module named 'libsbml'</details>                                                                                                                                                                                                                                                                                                                                                                                             | <details><summary>Details</summary>```Command '-i /root/in/LEMS_NML2_Ex9_FN_missing_xmlns.omex -o /root/out' in image 'ghcr.io/biosimulators/xpp' returned non-zero exit status 1```</details>       |                                                            |                                                           |